## Notebook for making predictions with deleted constant y samples


In [167]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [168]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [169]:
X_A_train = dm.data_A.iloc[:, 2:-1]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:-1]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:-1]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

Y_A_train


0         0.00
1         0.00
2         0.00
3         0.00
4        19.36
         ...  
23219     9.02
23220     0.00
23221     0.00
23222     0.00
23223     0.00
Name: pv_measurement, Length: 23224, dtype: float64

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [170]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

#catModel_bc = CatBoostRegressor()

In [171]:
X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

In [172]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

#catModel_bc.fit(X_BC_train,Y_BC_train)


Learning rate set to 0.067297
0:	learn: 1245.8177147	total: 6.27ms	remaining: 6.26s
1:	learn: 1184.1122592	total: 10.4ms	remaining: 5.21s
2:	learn: 1126.7861233	total: 14.9ms	remaining: 4.94s
3:	learn: 1074.5039607	total: 18.9ms	remaining: 4.71s
4:	learn: 1025.9485387	total: 23ms	remaining: 4.58s
5:	learn: 980.6451839	total: 27ms	remaining: 4.47s
6:	learn: 940.9041303	total: 31.8ms	remaining: 4.52s
7:	learn: 903.5522763	total: 35.8ms	remaining: 4.44s
8:	learn: 868.8090291	total: 39.6ms	remaining: 4.36s
9:	learn: 838.0931980	total: 43.6ms	remaining: 4.32s
10:	learn: 809.2516491	total: 47.6ms	remaining: 4.28s
11:	learn: 783.6050462	total: 51.4ms	remaining: 4.24s
12:	learn: 759.1809175	total: 55.2ms	remaining: 4.19s
13:	learn: 737.4384392	total: 59ms	remaining: 4.15s
14:	learn: 717.4159867	total: 62.7ms	remaining: 4.12s
15:	learn: 699.1959899	total: 66.8ms	remaining: 4.11s
16:	learn: 683.7493414	total: 71.2ms	remaining: 4.12s
17:	learn: 668.6702165	total: 75.1ms	remaining: 4.1s
18:	learn:

## Preds

In [173]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

#pred_B = catModel_bc.predict(X_B_test)
#pred_C = catModel_bc.predict(X_C_test)

In [174]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/catboost_bc_1.csv")

df.shape

(2160, 1)